# HEC-RAS 1D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [1]:
# Install ras-commander from pip (uncomment to install if needed)
!pip install --upgrade ras-commander
# This installs ras-commander and all dependencies

# Set to false to disable plot generation for llm-friendly outputs
generate_plots = True

In [2]:
# Import all required modules
from ras_commander import *  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import scipy
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython import display
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil
from datetime import datetime, timedelta
from pathlib import Path  # Ensure pathlib is imported for file operations
from shapely.geometry import LineString


# Set pandas display options to show only 7 rows by default
pd.set_option('display.max_rows', 7)

# Use Example Project or Load Your Own Project

In [3]:
# Download the Balde Eagle Creek 1D Example project from HEC and run plan 01

# Define the path to the 1D Balde Eagle Creek project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "Balde Eagle Creek"
import logging

# Check if BaldEagle.p01.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagle.p01.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    RasExamples.extract_project("Balde Eagle Creek")

    # Initialize the RAS project using the custom ras object
    init_ras_project(bald_eagle_path, "6.6")
    logging.info(f"Balde Eagle project initialized with folder: {ras.project_folder}")
    
    logging.info(f"Balde Eagle object id: {id(ras)}")
    
    # Define the plan number to execute
    plan_number = "01"

    # Execute Plan 01 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagle.p01.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    init_ras_project(bald_eagle_path, "6.6")
    plan_number = "01"

2025-04-09 07:24:48 - ras_commander.RasPrj - INFO - Initializing global 'ras' object via init_ras_project function.
2025-04-09 07:24:48 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap
2025-04-09 07:24:48 - ras_commander.RasPrj - INFO - Project initialized. ras_object project folder: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek


BaldEagle.p01.hdf already exists. Skipping project extraction and plan execution.


###  OPTIONAL: Use your own project instead

your_project_path = Path(r"D:\yourprojectpath")

init_ras_project(your_project_path, "6.6")
plan_number = "01"  # Plan number to use for this notebook 



### If you use this code cell, don't run the previous cell or change to markdown
### NOTE: Ensure the HDF Results file was generated by HEC-RAS Version 6.x or above

------

# Explore Project Dataframes using 'ras' Object

In [4]:
print("Plan DataFrame for the project:")
ras.plan_df

Plan DataFrame for the project:


['  plan_number unsteady_number geometry_number                     Plan Title  \\\n', '0          01              02              01  Unsteady with Bridges and Dam   \n', '1          02            None              01                Steady Flow Run   \n', '\n', '  Program Version Short Identifier                  Simulation Date  \\\n', '0            5.00     UnsteadyFlow    18FEB1999,0000,24FEB1999,0500   \n', '1             NaN        SteadyRun  02/18/1999,0000,02/24/1999,0500   \n', '\n', '  Computation Interval Mapping Interval Run HTab  ... UNET D2 Cores PS Cores  \\\n', '0                 2MIN            1HOUR        1  ...           0.0     None   \n', '1                 2MIN              NaN        1  ...           NaN     None   \n', '\n', '  DSS File Friction Slope Method  \\\n', '0      dss                     2   \n', '1      dss                     1   \n', '\n', '                                    HDF_Results_Path  Geom File  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...         01   \n', '1                                               None         01   \n', '\n', '                                           Geom Path Flow File  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...        02   \n', '1  d:\\GitHub\\ras-commander\\examples\\example_proje...        02   \n', '\n', '                                           Flow Path  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '1  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '\n', '                                           full_path  \n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...  \n', '1  d:\\GitHub\\ras-commander\\examples\\example_proje...  \n', '\n', '[2 rows x 26 columns]']

In [5]:
print("\nGeometry DataFrame for the project:")
ras.geom_df


Geometry DataFrame for the project:


['  geom_file geom_number                                          full_path  \\\n', '0       g01          01  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '\n', '                                            hdf_path  \n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...  ']

In [6]:
print("\nUnsteady DataFrame for the project:")
ras.unsteady_df


Unsteady DataFrame for the project:


['  unsteady_number                                          full_path  \\\n', '0              02  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '\n', '  geometry_number         Flow Title Program Version Use Restart  \\\n', '0            None  Flow Hydrograph 2            6.30           0   \n', '\n', '  Precipitation Mode       Wind Mode Met BC=Precipitation|Mode  \\\n', '0            Disable  No Wind Forces                      None   \n', '\n', '  Met BC=Evapotranspiration|Mode Met BC=Precipitation|Expanded View  \\\n', '0                           None                                  0   \n', '\n', '  Met BC=Precipitation|Constant Units Met BC=Precipitation|Gridded Source  \n', '0                               mm/hr                                 DSS  ']

In [7]:
print("\nBoundary Conditions DataFrame for the project:")
ras.boundaries_df 


Boundary Conditions DataFrame for the project:


['  unsteady_number  boundary_condition_number river_reach_name river_station  \\\n', '0              02                          1       Bald Eagle       Loc Hav   \n', '1              02                          2       Bald Eagle       Loc Hav   \n', '2              02                          3       Bald Eagle       Loc Hav   \n', '\n', '  storage_area_name pump_station_name          bc_type  hydrograph_type  \\\n', '0          138154.4                    Flow Hydrograph  Flow Hydrograph   \n', '1             81500                       Gate Opening             None   \n', '2           659.942                            Unknown             None   \n', '\n', '  Interval DSS Path  ...         Flow Title Program Version Use Restart  \\\n', '0    1HOUR           ...  Flow Hydrograph 2            6.30           0   \n', '1      NaN      NaN  ...  Flow Hydrograph 2            6.30           0   \n', '2      NaN           ...  Flow Hydrograph 2            6.30           0   \n', '\n', '  Precipitation Mode       Wind Mode  Met BC=Precipitation|Mode  \\\n', '0            Disable  No Wind Forces                       None   \n', '1            Disable  No Wind Forces                       None   \n', '2            Disable  No Wind Forces                       None   \n', '\n', '  Met BC=Evapotranspiration|Mode Met BC=Precipitation|Expanded View  \\\n', '0                           None                                  0   \n', '1                           None                                  0   \n', '2                           None                                  0   \n', '\n', '  Met BC=Precipitation|Constant Units Met BC=Precipitation|Gridded Source  \n', '0                               mm/hr                                 DSS  \n', '1                               mm/hr                                 DSS  \n', '2                               mm/hr                                 DSS  \n', '\n', '[3 rows x 29 columns]']

In [8]:
# Get HDF Results Entries (only present when results are present)
ras.get_hdf_entries()

['  plan_number unsteady_number geometry_number                     Plan Title  \\\n', '0          01              02              01  Unsteady with Bridges and Dam   \n', '\n', '  Program Version Short Identifier                Simulation Date  \\\n', '0            5.00     UnsteadyFlow  18FEB1999,0000,24FEB1999,0500   \n', '\n', '  Computation Interval Mapping Interval Run HTab  ... UNET D2 Cores PS Cores  \\\n', '0                 2MIN            1HOUR        1  ...           0.0     None   \n', '\n', '  DSS File Friction Slope Method  \\\n', '0      dss                     2   \n', '\n', '                                    HDF_Results_Path  Geom File  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...         01   \n', '\n', '                                           Geom Path Flow File  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...        02   \n', '\n', '                                           Flow Path  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '\n', '                                           full_path  \n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...  \n', '\n', '[1 rows x 26 columns]']

# Find Paths for Results and Geometry HDF's

In [9]:
# Get the plan HDF path for the plan_number defined above
plan_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

In [10]:
plan_hdf_path

["'d:\\\\GitHub\\\\ras-commander\\\\examples\\\\example_projects\\\\Balde Eagle Creek\\\\BaldEagle.p01.hdf'"]

In [11]:
# Get the geometry HDF path
geom_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'Geom Path'].values[0] + '.hdf'

In [12]:
geom_hdf_path

["'d:\\\\GitHub\\\\ras-commander\\\\examples\\\\example_projects\\\\Balde Eagle Creek\\\\BaldEagle.g01.hdf'"]

In [13]:
print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")


Plan HDF path for Plan 01: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
Geometry HDF path for Plan 01: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf


## RAS-Commander's Decorators Allow for Flexible Function Calling
You can call most of the functions in the HDF* Classes using any of the following:
1. Plan/Geometry Number (with or without leading zeros):
   - "01", "1" - Plan/geometry number as string
   - 1 - Plan/geometry number as integer
   - "p01", "p1" - Plan number with 'p' prefix
2. Direct File Paths:
   - pathlib.Path object pointing to HDF file
   - String path to HDF file

3. h5py.File Objects:
   - Already opened HDF file object

The @standardize_input decorator handles all these input types consistently:
   - Validates the input exists and is accessible
   - Converts to proper pathlib.Path object
   - Handles RAS object references
   - Provides logging and error handling

This flexibility makes it easier to work with HDF files in different contexts while maintaining consistent behavior 
across the codebase, and helps prevent strict typing from introducing unnecessary friction for LLM Coding.


-----

# 1D HDF Data Extraction Examples

In [14]:
# Extract runtime and compute time data as dataframe
print("\nExtracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_path=plan_number)

2025-04-09 07:24:49 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:49 - ras_commander.HdfResultsPlan - INFO - Extracting Plan Information from: BaldEagle.p01.hdf
2025-04-09 07:24:49 - ras_commander.HdfResultsPlan - INFO - Plan Name: Unsteady with Bridges and Dam
2025-04-09 07:24:49 - ras_commander.HdfResultsPlan - INFO - Simulation Duration (hours): 149.0



Extracting runtime and compute time data


In [15]:
runtime_df

['                       Plan Name          File Name Simulation Start Time  \\\n', '0  Unsteady with Bridges and Dam  BaldEagle.p01.hdf            1999-02-18   \n', '\n', '  Simulation End Time  Simulation Duration (s)  Simulation Time (hr)  \\\n', '0 1999-02-24 05:00:00                 536400.0                 149.0   \n', '\n', '  Completing Geometry (hr)  Preprocessing Geometry (hr)  \\\n', '0                      N/A                     0.042926   \n', '\n', '  Completing Event Conditions (hr)  Unsteady Flow Computations (hr)  \\\n', '0                              N/A                         0.001098   \n', '\n', '   Complete Process (hr)  Unsteady Flow Speed (hr/hr)  \\\n', '0               0.049271                135694.409309   \n', '\n', '   Complete Process Speed (hr/hr)  \n', '0                      3024.10148  ']

In [16]:
# Use HdfUtils for extracting projection
# This returns a string with the projection as EPSG code (e.g. "EPSG:6556"), or None if not found.
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)  
# This projection is returned as EPSG to improve compatibility with geopandas

2025-04-09 07:24:49 - ras_commander.HdfBase - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBase - CRITICAL - No valid projection found. Checked:
1. HDF file projection attribute: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
 was checked and no projection attribute found2. No RASMapper projection file found
To fix this:
1. Open RASMapper
2. Click Map > Set Projection
3. Select an appropriate projection file or coordinate system
4. Save the RASMapper project



Extracting Projection from HDF


In [17]:
projection
### The example project we are using does not have a projection  

In [18]:
# Use HdfPlan to Get Geometry Information (Base Geometry Attributes) as dataframes
print("\nExtracting Base Geometry Attributes")
geom_attrs_df = HdfPlan.get_geometry_information("01")  
# NOTE: Here we call the function using the plan number instead of the hdf path to demonstrate that the decorator will work with the plan number


2025-04-09 07:24:49 - ras_commander.RasPlan - INFO - Found geometry path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Found geometry HDF file for plan 1: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Getting geometry attributes from d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Checking for Geometry group in d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Getting root level geometry attributes
2025-04-09 07:24:49 - ras_commander.HdfPlan - INFO - Successfully


Extracting Base Geometry Attributes


In [19]:
geom_attrs_df

['                                                               Value\n', 'Complete Geometry                                               True\n', 'Extents            [1966632.2513, 2066299.3787, 287473.1387, 3540...\n', 'Geometry Time                                    2025-04-09 07:13:45\n', 'Title                                 Existing Conditions - GIS Data\n', 'Version                                           1.0.20 (20Sep2024)']

In [20]:
# Get geometry structures attributes as dataframe
print("\nGetting geometry structures attributes")
geom_structures_attrs_df = HdfStruc.get_geom_structures_attrs(geom_hdf_path)

2025-04-09 07:24:49 - ras_commander.HdfStruc - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfStruc - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf



Getting geometry structures attributes


In [21]:
geom_structures_attrs_df

['   Bridge/Culvert Count  Connection Count  Has Bridge Opening (2D)  \\\n', '0                    10                 0                        0   \n', '\n', '   Inline Structure Count  Lateral Structure Count  \n', '0                       1                        0  ']

In [22]:
# Instead of hdf_input, USE plan_hdf_path or geom_hdf_path, or the plan number as "8" or "08" 
# Input decorators allow for flexible inputs 

In [23]:
# Get structures as geodataframe
structures_gdf = HdfStruc.get_structures(geom_hdf_path)

2025-04-09 07:24:49 - ras_commander.HdfStruc - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfStruc - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBase - CRITICAL - No valid projection found. Checked:
1. HDF file projection attribute: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
 was checked and no projection attribute found2. No RASMapper projection file found
To fix

In [24]:
structures_gdf

['      Type Mode       River    Reach      RS Connection  \\\n', '0   Bridge       Bald Eagle  Loc Hav  103245              \n', '1   Inline       Bald Eagle  Loc Hav   81500              \n', '2   Bridge       Bald Eagle  Loc Hav   75960              \n', '..     ...  ...         ...      ...     ...        ...   \n', '8   Bridge       Bald Eagle  Loc Hav   11985              \n', '9   Bridge       Bald Eagle  Loc Hav    2920              \n', '10  Bridge       Bald Eagle  Loc Hav    2436              \n', '\n', '                       Groupname US Type    US River US Reach  ...  \\\n', '0   Bald Eagle, Loc Hav (103245)      XS  Bald Eagle  Loc Hav  ...   \n', '1    Bald Eagle, Loc Hav (81500)      XS  Bald Eagle  Loc Hav  ...   \n', '2    Bald Eagle, Loc Hav (75960)      XS  Bald Eagle  Loc Hav  ...   \n', '..                           ...     ...         ...      ...  ...   \n', '8    Bald Eagle, Loc Hav (11985)      XS  Bald Eagle  Loc Hav  ...   \n', '9     Bald Eagle, Loc Hav (2
...
[Output truncated, 3172 characters total]

In [25]:
# Get reference lines as geodataframe
ref_lines_gdf = HdfBndry.get_reference_lines(geom_hdf_path)

2025-04-09 07:24:49 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBndry - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf


In [26]:
ref_lines_gdf

""
[... additional rows truncated ...]


In [27]:
# Get reference points as geodataframe
ref_points_gdf = HdfBndry.get_reference_points(geom_hdf_path)

2025-04-09 07:24:49 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:49 - ras_commander.HdfBndry - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf


In [28]:
ref_points_gdf

""
[... additional rows truncated ...]


In [29]:
# Get cross sections as geodataframe
cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path)
    

In [30]:
cross_sections_gdf

['                                              geometry  \\\n', '0    LINESTRING (1968668.17 290166.79, 1969067.87 2...   \n', '1    LINESTRING (1968627.02 290584.12, 1969009.09 2...   \n', '2    LINESTRING (1968585.88 290854.5, 1968868.02 29...   \n', '..                                                 ...   \n', '175  LINESTRING (2063077.13 352972.51, 2063714.94 3...   \n', '176  LINESTRING (2063956.46 353190.59, 2064617.71 3...   \n', '177  LINESTRING (2064505.16 353331.28, 2065091.38 3...   \n', '\n', '                                     station_elevation  \\\n', '0    [[0.0, 660.41], [5.0, 660.61], [40.0, 659.85],...   \n', '1    [[0.0, 664.28], [50.0, 661.73], [55.0, 661.54]...   \n', '2    [[0.0, 662.72], [20.0, 665.5], [25.0, 666.48],...   \n', '..                                                 ...   \n', '175  [[0.0, 548.4], [130.0, 547.37], [215.0, 546.24...   \n', '176  [[0.0, 547.93], [75.0, 547.5], [145.01, 547.76...   \n', '177  [[0.0, 539.78], [5.0, 539.74], [40.0, 53
...
[Output truncated, 3915 characters total]

In [31]:
# Showing only cross sections with ineffective flow areas

# Filter rows where ineffective_blocks is not empty
ineffective_xs_gdf = cross_sections_gdf[cross_sections_gdf['ineffective_blocks'].apply(len) > 0]
print("\nCross Sections with Ineffective Flow Areas:")


Cross Sections with Ineffective Flow Areas:


In [32]:
ineffective_xs_gdf

['                                              geometry  \\\n', '81   LINESTRING (2009787.46 325935.78, 2010709.8 32...   \n', '82   LINESTRING (2009989.76 325990.64, 2010493.79 3...   \n', '100  LINESTRING (2022341.9 333784.25, 2023231.41 33...   \n', '..                                                 ...   \n', '160  LINESTRING (2054283.09 350389.32, 2055114.51 3...   \n', '173  LINESTRING (2061688.97 352001.74, 2063053.69 3...   \n', '174  LINESTRING (2061670.21 352090.84, 2063025.55 3...   \n', '\n', '                                     station_elevation  \\\n', '81   [[0.0, 603.79], [2.0, 603.17], [4.0, 602.65], ...   \n', '82   [[0.0, 611.14], [6.0, 611.11], [8.0, 611.06], ...   \n', '100  [[0.0, 600.93], [5.0, 601.88], [35.0, 601.2], ...   \n', '..                                                 ...   \n', '160  [[0.0, 574.48], [40.0, 574.08], [45.0, 574.31]...   \n', '173  [[90.0, 556.52], [100.5, 557.67], [107.5, 557....   \n', '174  [[0.0, 551.6], [91.0, 551.38], [129.51, 
...
[Output truncated, 4273 characters total]

In [33]:
# Print first 5 cross sections data
print("\nCross Section Information:")

for idx, row in cross_sections_gdf.head(5).iterrows():
    print(f"\nCross Section {idx + 1}:")
    print(f"River: {row['River']}")
    print(f"Reach: {row['Reach']}")
    print("\nGeometry:")
    print(row['geometry'])
    print("\nStation-Elevation Points:")
    
    # Print header
    print("     #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation")
    print("-" * 150)
    
    # Calculate number of rows needed
    points = row['station_elevation']
    num_rows = (len(points) + 4) // 5  # Round up division
    
    # Print points in 5 columns
    for i in range(num_rows):
        line = ""
        for j in range(5):
            point_idx = i + j * num_rows
            if point_idx < len(points):
                station, elevation = points[point_idx]
                line += f"{point_idx+1:6d} {station:10.2f} {elevation:10.2f}    "
        print(line)
    print("-" * 150)



Cross Section Information:

Cross Section 1:
River: Bald Eagle
Reach: Loc Hav

Geometry:
LINESTRING (1968668.17 290166.79, 1969067.87 289778.85)

Station-Elevation Points:
     #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation
------------------------------------------------------------------------------------------------------------------------------------------------------
     1       0.00     660.41         9     190.00     660.21        17     330.00     660.95        25     410.00     671.32        33     480.00     695.50    
     2       5.00     660.61        10     195.00     658.98        18     340.00     659.56        26     415.00     672.91        34     485.00     696.55    
     3      40.00     659.85        11     197.96     657.39        19     345.00     658.84        27     430.00     678.12        35     514.99     701.46    
     4      55.00     659

In [34]:
# Plot cross sections on map with matplotlib

if generate_plots:
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(15,10))
    
    # Plot cross sections
    cross_sections_gdf.plot(ax=ax, color='red', linewidth=1, label='Cross Sections')
    
    # Add river name and reach labels
    #for idx, row in cross_sections_gdf.iterrows():
    #    # Get midpoint of cross section line for label placement
    #    midpoint = row.geometry.centroid
    #    label = f"{row['River']}\n{row['Reach']}\nRS: {row['RS']}"
    #    ax.annotate(label, (midpoint.x, midpoint.y), 
    #               xytext=(5, 5), textcoords='offset points',
    #               fontsize=8, bbox=dict(facecolor='white', alpha=0.7))
    
    # Customize plot
    ax.set_title('Cross Sections Location Map')
    ax.grid(True)
    ax.legend()
    
    # Equal aspect ratio to preserve shape
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

<Figure size 1500x1000 with 1 Axes>

In [35]:
# Plot cross sections with Manning's n values colored by value

if generate_plots:
    # Create figure
    fig, ax1 = plt.subplots(figsize=(20,10))

    # Create colormap
    cmap = plt.cm.viridis
    norm = plt.Normalize(vmin=0.02, vmax=0.08)  # Typical Manning's n range

    # Plot cross sections colored by Manning's n
    for idx, row in cross_sections_gdf.iterrows():
        # Extract Manning's n values and stations
        mannings = row['mannings_n']
        n_values = mannings['Mann n']
        stations = mannings['Station']
        
        # Get the full linestring coordinates
        line_coords = list(row.geometry.coords)
        
        # Calculate total length of the cross section
        total_length = row.geometry.length
        
        # For each Manning's n segment
        for i in range(len(n_values)-1):
            # Calculate the start and end proportions along the line
            start_prop = stations[i] / stations[-1]
            end_prop = stations[i+1] / stations[-1]
            
            # Get the start and end points for this segment
            start_idx = int(start_prop * (len(line_coords)-1))
            end_idx = int(end_prop * (len(line_coords)-1))
            
            # Extract the segment coordinates
            segment_coords = line_coords[start_idx:end_idx+1]
            
            if len(segment_coords) >= 2:
                # Create a line segment
                segment = LineString(segment_coords)
                
                # Get color from colormap for this n value
                color = cmap(norm(n_values[i]))
                
                # Plot the segment
                ax1.plot(*segment.xy, color=color, linewidth=2)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax1, label="Manning's n Value")

    ax1.set_title("Cross Sections Colored by Manning's n Values")
    ax1.grid(True)
    ax1.set_aspect('equal')

    plt.tight_layout()
    plt.show()

<Figure size 2000x1000 with 2 Axes>

In [36]:
# Plot cross sections with ineffective flow areas

if generate_plots:
    # Create figure
    fig, ax2 = plt.subplots(figsize=(20,10))

    # Plot all cross sections first
    cross_sections_gdf.plot(ax=ax2, color='lightgray', linewidth=1, label='Cross Sections')

    # Plot ineffective flow areas with thicker lines
    ineffective_sections = cross_sections_gdf[cross_sections_gdf['ineffective_blocks'].apply(lambda x: len(x) > 0)]
    ineffective_sections.plot(ax=ax2, color='red', linewidth=3, label='Ineffective Flow Areas')

    # Add ineffective flow area labels with offset to lower right
    for idx, row in cross_sections_gdf.iterrows():
        # Get midpoint of cross section line
        midpoint = row.geometry.centroid
        
        # Extract ineffective flow blocks
        ineff_blocks = row['ineffective_blocks']
        
        if ineff_blocks:  # Only label if there are ineffective blocks
            label_parts = []
            # Add RS to first line of label
            label_parts.append(f"RS: {row['RS']}")
            for block in ineff_blocks:
                label_parts.append(
                    f"L:{block['Left Sta']:.0f}-R:{block['Right Sta']:.0f}\n"
                    f"Elev: {block['Elevation']:.2f}\n"
                    f"Permanent: {block['Permanent']}"
                )
            
            label = '\n'.join(label_parts)
            
            ax2.annotate(label, (midpoint.x, midpoint.y),
                        xytext=(15, -15),  # Offset to lower right
                        textcoords='offset points',
                        fontsize=8, 
                        bbox=dict(facecolor='white', alpha=0.7),
                        arrowprops=dict(arrowstyle='->'),
                        horizontalalignment='left',
                        verticalalignment='top')

    ax2.set_title('Cross Sections with Ineffective Flow Areas')
    ax2.grid(True)
    ax2.legend()
    ax2.set_aspect('equal')

    plt.tight_layout()
    plt.show()

<Figure size 2000x1000 with 1 Axes>

In [37]:
# Plot cross section elevation for cross section 42
if generate_plots:
    # Get cross sections data
    cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path)

    if not cross_sections_gdf.empty:
        # Get station-elevation data for cross section 42
        station_elevation = cross_sections_gdf.iloc[42]['station_elevation']
        
        # Convert list of lists to numpy arrays for plotting
        stations = np.array([point[0] for point in station_elevation])
        elevations = np.array([point[1] for point in station_elevation])
        
        # Create figure and axis
        fig, ax = plt.subplots(figsize=(12,8))
        
        # Plot cross section
        ax.plot(stations, elevations, 'b-', linewidth=2)
        
        # Add labels and title
        river = cross_sections_gdf.iloc[42]['River']
        reach = cross_sections_gdf.iloc[42]['Reach'] 
        rs = cross_sections_gdf.iloc[42]['RS']
        
        # Show bank stations as dots
        left_bank_station = cross_sections_gdf.iloc[42]['Left Bank']
        right_bank_station = cross_sections_gdf.iloc[42]['Right Bank']
        
        # Get elevations at bank stations
        left_bank_elev = elevations[np.searchsorted(stations, left_bank_station)]
        right_bank_elev = elevations[np.searchsorted(stations, right_bank_station)]
        
        # Plot bank stations with dots
        ax.plot(left_bank_station, left_bank_elev, 'ro')
        ax.plot(right_bank_station, right_bank_elev, 'ro')
        
        # Add bank station labels with station and elevation
        ax.annotate(f'Left Bank\nStation: {left_bank_station:.1f}\nElevation: {left_bank_elev:.1f}',
                   (left_bank_station, left_bank_elev),
                   xytext=(-50, 30),
                   textcoords='offset points',
                   bbox=dict(facecolor='white', alpha=0.8),
                   arrowprops=dict(arrowstyle='->'))
                   
        ax.annotate(f'Right Bank\nStation: {right_bank_station:.1f}\nElevation: {right_bank_elev:.1f}',
                   (right_bank_station, right_bank_elev), 
                   xytext=(50, 30),
                   textcoords='offset points',
                   bbox=dict(facecolor='white', alpha=0.8),
                   arrowprops=dict(arrowstyle='->'))
        
        ax.set_title(f'Cross Section Profile\nRiver: {river}, Reach: {reach}, RS: {rs}')
        ax.set_xlabel('Station (ft)')
        ax.set_ylabel('Elevation (ft)')
        
        # Add grid
        ax.grid(True)
        
        plt.tight_layout()
        plt.show()


<Figure size 1200x800 with 1 Axes>

In [38]:
# Get river centerlines as geodataframe
centerlines_gdf = HdfXsec.get_river_centerlines(geom_hdf_path)

2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_proj

In [39]:
print("\nRiver Centerlines:")
centerlines_gdf


River Centerlines:


['   River Name Reach Name   US Type US Name   DS Type DS Name  \\\n', '0  Bald Eagle    Loc Hav  External          External           \n', '\n', '   Junction to US XS  DS XS to Junction  \\\n', '0                NaN                NaN   \n', '\n', '                                            geometry         length  \n', '0  LINESTRING (1967609.38 288126.34, 1967754.63 2...  140451.729879  ']

In [40]:
# Plot river centerlines with labels
if generate_plots:
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(15, 10))

    # Plot centerlines
    centerlines_gdf.plot(ax=ax, color='blue', linewidth=2, label='River Centerline')

    # Add river/reach labels
    for idx, row in centerlines_gdf.iterrows():
        # Get midpoint of the line for label placement
        midpoint = row.geometry.interpolate(0.5, normalized=True)
        
        # Create label text combining river and reach names
        label = f"{row['River Name']}\n{row['Reach Name']}"
        
        # Add text annotation
        ax.annotate(label, 
                    xy=(midpoint.x, midpoint.y),
                    xytext=(10, 10), # Offset text slightly
                    textcoords='offset points',
                    fontsize=10,
                    bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

    # Add labels and title
    ax.set_title('River Centerlines', fontsize=14)
    ax.set_xlabel('Easting', fontsize=12)
    ax.set_ylabel('Northing', fontsize=12)

    # Add legend
    ax.legend(fontsize=12)

    # Add grid
    ax.grid(True)

    # Adjust layout
    plt.tight_layout()

    # Show plot
    plt.show()



<Figure size 1500x1000 with 1 Axes>

In [41]:
# Get river edge lines as geodataframe
edge_lines_gdf = HdfXsec.get_river_edge_lines(geom_hdf_path)


2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_proj

In [42]:
print("\nRiver Edge Lines:")
edge_lines_gdf


River Edge Lines:


['   edge_id  bank_side                                           geometry  \\\n', '0      NaN        NaN  LINESTRING (1968668.17 290166.79, 1968673.632 ...   \n', '1      NaN        NaN  LINESTRING (1969067.87 289778.85, 1969194.669 ...   \n', '\n', '          length  \n', '0  141615.154108  \n', '1  130396.337697  ']

In [43]:
# Get bank lines as geodataframe
bank_lines_gdf = HdfXsec.get_river_bank_lines(geom_hdf_path)


2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfXsec - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01.hdf
2025-04-09 07:24:52 - ras_commander.HdfBase - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_proj

In [44]:
print("\nRiver Bank Lines:")
bank_lines_gdf


River Bank Lines:


['   bank_id bank_side                                           geometry  \\\n', '0        0      Left  LINESTRING (1968804.51 290034.461, 1968930.093...   \n', '1        1     Right  LINESTRING (1968937.263 289905.614, 1969101.06...   \n', '\n', '          length  \n', '0  139546.829503  \n', '1  139349.187666  ']

In [45]:
# Create figure and axis

if generate_plots:
    fig, ax = plt.subplots(figsize=(15, 10))

    # Plot river edge lines
    edge_lines_gdf.plot(ax=ax, color='blue', linewidth=2, label='River Edge Lines')

    # Plot centerlines for reference
    centerlines_gdf.plot(ax=ax, color='red', linewidth=2, linestyle='--', label='River Centerline')

    # Plot river bank lines
    bank_lines_gdf.plot(ax=ax, color='green', linewidth=2, label='River Bank Lines')

    # Add title and labels
    ax.set_title('River Edge Lines, Centerline, and Bank Lines', fontsize=14)
    ax.set_xlabel('Easting', fontsize=12)
    ax.set_ylabel('Northing', fontsize=12)

    # Add legend
    ax.legend(fontsize=12)

    # Add grid
    ax.grid(True)

    # Adjust layout
    plt.tight_layout()

    # Show plot
    plt.show()

<Figure size 1500x1000 with 1 Axes>

In [46]:
# Extract 1D Structures Geodataframe



# Display basic information about the structures
print("\nStructures Summary:")
print(f"Number of structures found: {len(structures_gdf)}")
structures_gdf

# Display first few rows of key attributes
print("\nStructure Details:")
display_cols = ['Structure ID', 'Structure Type', 'River Name', 'Reach Name', 'Station']
display_cols = [col for col in display_cols if col in structures_gdf.columns]
if display_cols:
    print(structures_gdf[display_cols].head())


if generate_plots:

    # Create visualization
    fig, ax = plt.subplots(figsize=(15, 10))

    # Plot river centerlines
    if not centerlines_gdf.empty:
        centerlines_gdf.plot(ax=ax, color='blue', linewidth=2, label='River Centerlines')

    # Plot cross sections
    if not cross_sections_gdf.empty:
        cross_sections_gdf.plot(ax=ax, color='green', linewidth=1, label='Cross Sections')

    # Plot structures
    if not structures_gdf.empty:
        structures_gdf.plot(ax=ax, color='red', marker='s', markersize=100, label='Structures')

    # Add title and labels
    ax.set_title('HEC-RAS Model Components', fontsize=14)
    ax.set_xlabel('Easting', fontsize=12)
    ax.set_ylabel('Northing', fontsize=12)

    # Add legend
    ax.legend(fontsize=12)

    # Add grid
    ax.grid(True)

    # Adjust layout
    plt.tight_layout()

    # Show plot
    plt.show()

# Print summary of cross sections
print("\nCross Sections Summary:")
print(f"Number of cross sections found: {len(cross_sections_gdf)}")
if not cross_sections_gdf.empty:
    print("\nCross Section Details:")
    xs_display_cols = ['River', 'Reach', 'Station']
    xs_display_cols = [col for col in xs_display_cols if col in cross_sections_gdf.columns]
    if xs_display_cols:
        print(cross_sections_gdf[xs_display_cols].head())



Structures Summary:
Number of structures found: 11

Structure Details:
   Structure ID
0             1
1             2
2             3
3             4
4             5


<Figure size 1500x1000 with 1 Axes>


Cross Sections Summary:
Number of cross sections found: 178

Cross Section Details:
        River    Reach
0  Bald Eagle  Loc Hav
1  Bald Eagle  Loc Hav
2  Bald Eagle  Loc Hav
3  Bald Eagle  Loc Hav
4  Bald Eagle  Loc Hav


In [47]:
# Extract Plan Parameters
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")

plan_parameters_df = HdfPlan.get_plan_parameters(hdf_path=plan_hdf_path)

2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf



Example 12: Extracting Plan Parameters and Volume Accounting Data


In [48]:
print("\nPlan Parameters DataFrame:")
plan_parameters_df


Plan Parameters DataFrame:


['   Plan              Parameter  Value\n', '0    01               1D Cores      0\n', '1    01      1D Flow Tolerance    NaN\n', '2    01  1D Maximum Iterations     20\n', '..  ...                    ...    ...\n', '29   01  HDF Write Time Slices  False\n', '30   01       HDF Write Warmup  False\n', '31   01         Pardiso Solver  False\n', '\n', '[32 rows x 3 columns]']

In [49]:
# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_volume_accounting(hdf_path=plan_hdf_path)

2025-04-09 07:24:53 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:53 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


In [50]:
print("\nVolume Accounting DataFrame:")
volume_accounting_df


Volume Accounting DataFrame:


['       Error  Error Percent  Total Boundary Flux of Water In  \\\n', '0 -29.546846       0.014071                    200682.453125   \n', '\n', '   Total Boundary Flux of Water Out Vol Accounting in  Volume Ending  \\\n', '0                     112574.828125         Acre Feet   97384.882812   \n', '\n', '   Volume Starting  \n', '0      9306.797852  ']

In [51]:
# Get simulation start time
start_time = HdfPlan.get_plan_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


Simulation start time: 1999-02-18 00:00:00


In [52]:
# Get plan end time
end_time = HdfPlan.get_plan_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:53 - ras_commander.HdfPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


Simulation end time: 1999-02-24 05:00:00


In [53]:
# Plot the time of maximum water surface elevation (WSEL) for cross sections

# Get cross section results timeseries
xsec_results_xr = HdfResultsXsec.get_xsec_timeseries(plan_hdf_path)
print("\nCross Section Results Shape:", xsec_results_xr['Water_Surface'].shape)

# Get cross section geometry data
xsec_geom = HdfXsec.get_cross_sections(plan_hdf_path)
print("\nNumber of cross sections in geometry:", len(xsec_geom))

# Create dataframe with cross section locations and max WSEL times
xs_data = []

# Extract water surface data from xarray Dataset
water_surface = xsec_results_xr['Water_Surface'].values
times = pd.to_datetime(xsec_results_xr.time.values)

# Debug print
print("\nFirst few cross section names:")
print(xsec_results_xr.cross_section.values[:5])

# Iterate through cross sections
for xs_idx in range(len(xsec_results_xr.cross_section)):
    # Get WSEL timeseries for this cross section
    wsel_series = water_surface[:, xs_idx]
    
    # Get cross section name and parse components
    xs_name = xsec_results_xr.cross_section.values[xs_idx]
    
    # Split the string and remove empty strings
    xs_parts = [part for part in xs_name.split() if part]
    
    if len(xs_parts) >= 3:
        river = "Bald Eagle"  # Combine first two words
        reach = "Loc Hav"     # Next two words
        rs = xs_parts[-1]     # Last part is the station
        
        # Get geometry for this cross section
        xs_match = xsec_geom[
            (xsec_geom['River'] == river) & 
            (xsec_geom['Reach'] == reach) & 
            (xsec_geom['RS'] == rs)
        ]
        
        if not xs_match.empty:
            geom = xs_match.iloc[0]
            # Use first point of cross section line for plotting
            x = geom.geometry.coords[0][0]
            y = geom.geometry.coords[0][1]
            
            # Find time of max WSEL
            max_wsel_idx = np.argmax(wsel_series)
            max_wsel = np.max(wsel_series)
            max_time = times[max_wsel_idx]
            
            xs_data.append({
                'xs_name': xs_name,
                'x': x,
                'y': y,
                'max_wsel': max_wsel,
                'time_of_max': max_time
            })
        else:
            print(f"\nWarning: No geometry match found for {xs_name}")
            print(f"River: {river}, Reach: {reach}, RS: {rs}")
    else:
        print(f"\nWarning: Could not parse cross section name: {xs_name}")

# Create dataframe
xs_df = pd.DataFrame(xs_data)

# Debug print
print("\nNumber of cross sections processed:", len(xs_df))




if generate_plots:
    print("\nColumns in xs_df:", xs_df.columns.tolist())
    print("\nFirst row of xs_df:")
    print(xs_df.iloc[0])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert datetime to hours since start for colormap
    min_time = min(xs_df['time_of_max'])
    color_values = [(t - min_time).total_seconds() / 3600 for t in xs_df['time_of_max']]

    # Plot cross section points
    scatter = ax.scatter(xs_df['x'], xs_df['y'],
                        c=color_values,
                        cmap='viridis',
                        s=50)

    # Customize plot
    ax.set_title('Time of Maximum Water Surface Elevation at Cross Sections')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Hours since simulation start')

    # Format colorbar ticks
    max_hours = int(max(color_values))
    tick_interval = max(1, max_hours // 6)  # Show ~6 ticks
    cbar.set_ticks(range(0, max_hours + 1, tick_interval))
    cbar.set_ticklabels([f'{h}h' for h in range(0, max_hours + 1, tick_interval)])

    # Add grid and adjust styling
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.rcParams.update({'font.size': 12})
    plt.tight_layout()

    # Show plot
    plt.show()

    # Print summary statistics
    max_wsel_xs = xs_df.loc[xs_df['max_wsel'].idxmax()]
    hours_since_start = (max_wsel_xs['time_of_max'] - min_time).total_seconds() / 3600

    print(f"\nOverall Maximum WSEL: {max_wsel_xs['max_wsel']:.2f} ft")
    print(f"Time of Overall Maximum WSEL: {max_wsel_xs['time_of_max']}")
    print(f"Hours since simulation start: {hours_since_start:.2f} hours")
    print(f"Location of Overall Maximum WSEL: X={max_wsel_xs['x']:.2f}, Y={max_wsel_xs['y']:.2f}")
    print(f"Cross Section: {max_wsel_xs['xs_name']}")


2025-04-09 07:24:53 - ras_commander.HdfResultsXsec - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:53 - ras_commander.HdfResultsXsec - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf



Cross Section Results Shape: (150, 178)

Number of cross sections in geometry: 178

First few cross section names:
['Bald Eagle       Loc Hav          138154.4'
 'Bald Eagle       Loc Hav          137690.8'
 'Bald Eagle       Loc Hav          137327.0'
 'Bald Eagle       Loc Hav          136564.9'
 'Bald Eagle       Loc Hav          136202.3']

Number of cross sections processed: 178

Columns in xs_df: ['xs_name', 'x', 'y', 'max_wsel', 'time_of_max']

First row of xs_df:
xs_name        Bald Eagle       Loc Hav          138154.4
x                                              1968668.17
y                                               290166.79
max_wsel                                       672.808289
time_of_max                           1999-02-19 15:00:00
Name: 0, dtype: object


<Figure size 1200x800 with 2 Axes>


Overall Maximum WSEL: 672.81 ft
Time of Overall Maximum WSEL: 1999-02-19 15:00:00
Hours since simulation start: 0.00 hours
Location of Overall Maximum WSEL: X=1968668.17, Y=290166.79
Cross Section: Bald Eagle       Loc Hav          138154.4


In [54]:
# Get unsteady attributes as dataframe
results_unsteady_attrs = HdfResultsPlan.get_unsteady_info(plan_hdf_path)

2025-04-09 07:24:54 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:54 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


In [55]:
results_unsteady_attrs

['                      Plan Title                     Program Name  \\\n', '0  Unsteady with Bridges and Dam  HEC-RAS - River Analysis System   \n', '\n', '              Program Version  \\\n', '0  HEC-RAS 6.6 September 2024   \n', '\n', '                                   Project File Name  \\\n', '0  d:\\GitHub\\ras-commander\\examples\\example_proje...   \n', '\n', '                         Project Title      Short ID  \\\n', '0  Bald Eagle Cr. for the WCDS Project  UnsteadyFlow   \n', '\n', '             Simulation Time Window             Type of Run  \n', '0  17Feb1999 2400 to 24Feb1999 0500  Unsteady Flow Analysis  ']

In [56]:
# Get unsteady summary attributes as dataframe
results_unsteady_summary_attrs = HdfResultsPlan.get_unsteady_summary(plan_hdf_path)

2025-04-09 07:24:54 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:54 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


In [57]:
results_unsteady_summary_attrs

['  Computation Time DSS Computation Time Total  Maximum WSEL Error  \\\n', '0             00:00:00               00:00:03            0.009995   \n', '\n', '   Maximum number of cores                           Run Time Window  \\\n', '0                        1  09APR2025 07:16:26 to 09APR2025 07:16:29   \n', '\n', '                         Solution  Time Solution Went Unstable  \\\n', '0  Unsteady Finished Successfully                          NaN   \n', '\n', '  Time Stamp Solution Went Unstable  \n', '0                    Not Applicable  ']

# 1D Cross Section Results as Xarray

In [58]:
# Get cross section results timeseries as xarray dataset
xsec_results_xr = HdfResultsXsec.get_xsec_timeseries(plan_hdf_path)

2025-04-09 07:24:54 - ras_commander.HdfResultsXsec - INFO - Using HDF file from direct string path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-04-09 07:24:54 - ras_commander.HdfResultsXsec - INFO - Final validated HDF file path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf


In [59]:
xsec_results_xr

xarray.Dataset with Unknown dimensions
[Full xarray output truncated during preprocessing]

In [60]:
# Print time series for specific cross section
target_xs = "Bald Eagle       Loc Hav          136202.3"

print("\nTime Series Data for Cross Section:", target_xs)
for var in ['Water_Surface', 'Velocity_Total', 'Velocity_Channel', 'Flow_Lateral', 'Flow']:
    print(f"\n{var}:")
    print(xsec_results_xr[var].sel(cross_section=target_xs).values[:5])  # Show first 5 values

# Create time series plots

if generate_plots:

    # Create a figure for each variable
    variables = ['Water_Surface', 'Velocity_Total', 'Velocity_Channel', 'Flow_Lateral', 'Flow']

    for var in variables:
        plt.figure(figsize=(10, 5))
        # Convert time values to datetime if needed
        time_values = pd.to_datetime(xsec_results_xr.time.values)
        values = xsec_results_xr[var].sel(cross_section=target_xs).values
        
        # Plot with explicit x and y values
        plt.plot(time_values, values, '-', linewidth=2)
        
        plt.title(f'{var} at {target_xs}')
        plt.xlabel('Time')
        plt.ylabel(var.replace('_', ' '))
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # Force display
        plt.draw()
        plt.pause(0.1)
        plt.show()



Time Series Data for Cross Section: Bald Eagle       Loc Hav          136202.3

Water_Surface:
[657.4173  656.7111  657.02997 657.44073 657.88824]

Velocity_Total:
[0.        2.7810786 2.9612713 2.9395368 3.0228102]

Velocity_Channel:
[0.        3.132834  3.4508243 3.7217746 4.083391 ]

Flow_Lateral:
[0. 0. 0. 0. 0.]

Flow:
[2000.     1273.8193 1607.1132 2109.992  2746.2964]


<Figure size 1000x500 with 1 Axes>

<Figure size 1000x500 with 1 Axes>

<Figure size 1000x500 with 1 Axes>

<Figure size 1000x500 with 1 Axes>

<Figure size 1000x500 with 1 Axes>

-----

# Advanced HDF Data Extraction
This section focuses on directly accessing the HDF file from a jupyter notebook for use cases not directly supported by the RAS-Commander libary:

In [61]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np

def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"

try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

Extracting Compute Messages
Compute Messages:
Plan                                     : 'Unsteady with Bridges and Dam' (BaldEagle.p01)
Simulation started at                    : 09Apr2025 07:13:47 AM

Writing Plan GIS Data...

Completed Writing Plan GIS Data

Writing Geometry...

Computing Bank Lines

Bank lines generated in 210 ms

Computing Edge Lines

Edge Lines generated in 91 ms

Computing XS Interpolation Surface

XS Interpolation Surface generated in 244 ms

Completed Writing Geometry

Writing Event Conditions ...

Completed Writing Event Condition Data

Geometric Preprocessor HEC-RAS 6.6 September 2024

Finished Processing Geometry

Performing Unsteady Flow Simulation  HEC-RAS 6.6 September 2024
Unsteady Input Summary                   : 

1D Unsteady Finite Difference Numerical Solution
Overall Volume Accounting Error in Acre Feet : -29.5468461514
Overall Volume Accounting Error as percentage : 0.01407

Please review "Computational Log File" output for volume accounting deta

## Exploring HDF Datasets with HdfBase.get_dataset_info
This allows users to find HDF information that is not included in the ras-commander library.  Find the path in HDFView and set the group_path below to explore the HDF datasets and attributes.  Then, use the output to write your own function to extract the data.  

# Get HDF Paths with Properties (For Exploring HDF Files)
HdfBase.get_dataset_info(plan_number, group_path="/Geometry")

#### Use get_hdf5_dataset_info function to get dataset structure:
HdfBase.get_dataset_info(plan_hdf_path, "/Geometry/River Bank Lines/")

#### Use get_hdf5_dataset_info function to get Pipe Conduits data:
HdfBase.get_dataset_info(plan_hdf_path, "/Geometry/Structures")


#### Use get_hdf5_dataset_info function to get Pipe Conduits data:
HdfBase.get_dataset_info(plan_hdf_path, "/Results/Unsteady/Output/Output Blocks/Computation Block/Global/")

#### Use the get_hdf5_dataset_info function from HdfUtils to explore the Cross Sections structure in the geometry HDF file

print("\nExploring Cross Sections structure in geometry file:")
print("HDF Base Path: /Geometry/Cross Sections ")
HdfBase.get_dataset_info(geom_hdf_path, group_path='/Geometry/Cross Sections')

print("\n=== HDF5 File Structure ===\n")
print(plan_hdf_path)
HdfBase.get_dataset_info(plan_hdf_path, group_path='/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Cross Sections')

For HDF datasets that are not supported by the RAS-Commadner library, provide the dataset path to HdfBase.get_dataset_info and provide the output to an LLM along with a relevent HDF* class(es) to generate new functions that extend the library's coverage.   